Data Class

In [2]:
import random

class Sentiment:
    NEGATIVE = "NEGATIVE"
    POSITIVE = "POSITIVE"
    NEUTRAL = "NEUTRAL"

class Review:
    def __init__(self,text,score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
    
    def get_sentiment(self):
        if(self.score <= 2):
            return Sentiment.NEGATIVE
        elif(self.score <= 3):
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE

class ReviewContainer:
    def __init__(self,reviews):
        self.reviews = reviews

    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]

    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))

        positive_shunk = positive[:len(negative)]
        self.reviews = negative + positive_shunk
        random.shuffle(self.reviews)
        

Data Loading

In [3]:
import json
file_name = 'Books_small_10000.jsonl'

reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

print(reviews[5].score)
print(reviews[5].text)
print(reviews[5].sentiment)


5.0
I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia's trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character's voice on a strong subject and making it so that other peoples story may be heard through Mia's.
POSITIVE


Data Prep

In [4]:
from sklearn.model_selection import train_test_split
training,test = train_test_split(reviews,test_size=0.33,random_state=42)

train_container = ReviewContainer(training)
train_container.evenly_distribute()

test_container = ReviewContainer(test)
test_container.evenly_distribute()

In [5]:
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

436
436


Bag of words vectorization

In [6]:
from  sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

vectorizer = TfidfVectorizer()

train_x_vector = vectorizer.fit_transform(train_x)

test_x_vector = vectorizer.transform(test_x)

print(vectorizer.get_feature_names_out())
print(train_x[0])
print(train_x_vector[0].toarray())


['00' '000' '01' ... 'zombies' 'zone' 'zora']
Enjoyed.... lots of subterfuge. Which sometimes got a little tedious for me. Overall, good plot and characters. I Like women in charge!
[[0. 0. 0. ... 0. 0. 0.]]


Classification

Support Vector Machine(SVM)

In [19]:
from sklearn import svm

clf_svm = svm.SVC(C = 4, kernel="rbf")

clf_svm.fit(train_x_vector,train_y)

clf_svm.predict(test_x_vector[0])


array(['POSITIVE'], dtype='<U8')

Decision Tree

In [8]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()

clf_dec.fit(train_x_vector,train_y)

clf_dec.predict(test_x_vector[0])

array(['POSITIVE'], dtype='<U8')

Naive Bayes

In [9]:
from sklearn.naive_bayes import GaussianNB

clf_nb = GaussianNB()

clf_nb.fit(train_x_vector.toarray(),train_y)

clf_nb.predict(test_x_vector[0].toarray().reshape(1,-1))

array(['NEGATIVE'], dtype='<U8')

Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()

clf_log.fit(train_x_vector,train_y)

clf_log.predict(train_x_vector[0])

array(['POSITIVE'], dtype='<U8')

Evaluation

In [14]:
# Mean Accurancy
print(clf_svm.score(test_x_vector,test_y))
print(clf_dec.score(test_x_vector,test_y))
print(clf_nb.score(test_x_vector.toarray(), test_y))
print(clf_log.score(test_x_vector,test_y))


0.8076923076923077
0.6442307692307693
0.6610576923076923
0.8028846153846154


In [15]:
# F1 Score
from sklearn.metrics import f1_score

print(f1_score(test_y,clf_svm.predict(test_x_vector),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
print(f1_score(test_y,clf_dec.predict(test_x_vector),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
print(f1_score(test_y,clf_nb.predict(test_x_vector.toarray()),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
print(f1_score(test_y,clf_log.predict(test_x_vector),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))

[0.80582524 0.80952381]
[0.63       0.65740741]
[0.65693431 0.66508314]
[0.80097087 0.8047619 ]


In [13]:
test_set = ['Very good', "bad book do not buy", 'fucking good']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'POSITIVE'], dtype='<U8')

Tunning the model (with Grid Search)

In [16]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear','rbf'), 'C':(1,4,8,16,32)}

svc = svm.SVC()
clf = GridSearchCV(svc,parameters,cv = 5)
clf.fit(train_x_vector,train_y)

,estimator,SVC()
,param_grid,"{'C': (1, ...), 'kernel': ('linear', ...)}"
,scoring,None
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,C,4


In [20]:
print(clf_svm.score(test_x_vector,test_y))

0.8197115384615384
